# Building the Final Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
%matplotlib inline
warnings.filterwarnings('ignore')

In [1]:
final_model = pd.read_csv('reddit.csv')
final_model.head()

<IPython.core.display.Javascript object>

,Unnamed: 0,Title,Upvote_ratio,Score,Gilded,Over_18,Number_of_Comments
0,0,House impeaches Trump for second time over Cap...,0.72,39464,2,False,8801
1,1,I'm a 16 yr old blacksmith and I forged this w...,0.90,29423,0,False,443
2,2,Do some research before spouting nonsense,0.90,50420,0,False,1008
3,3,Moving boxes arrive at the White House,0.81,149858,6,False,4392
4,4,“No One Took Us Seriously”: Black Cops Warned ...,0.98,7768,0,False,157


In [2]:
final_model.shape

(7471, 7)

In [3]:
#Dropping the unnamed column
final_model.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
#Dropping duplicate columns

final_model.drop_duplicates(keep = 'first', inplace = True)

In [5]:
final_model.columns

Index(['Title', 'Upvote_ratio', 'Score', 'Gilded', 'Over_18',
       'Number_of_Comments'],
      dtype='object')

In [6]:
#NUMBER OF WORDS IN THE TITLE

import re
def word_count(text):
    wordList = re.sub("[^\w]", " ",  text).split()
    return len(wordList)

final_model['word_count'] = final_model['Title'].map(lambda x : word_count(x))

## Cleaning the title column

In [8]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [9]:
#Downloading stopwords from NLTK
#We don't remove No and Not as it will help in sentiment analysis
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove("not")
stop_words.remove("no")

<IPython.core.display.Javascript object>

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
#Enforcing proper spacing

def spaceURLs(df, column):
    '''This method is used to enforce proper spacing
    Ex: In the data, you may have '[the image](https://image.xyz)';
    this method creates space between alt text ("the image") and the URL.'''
    data = df.copy()
    data.loc[:, column] = data[column].str.replace('\[|\]', ' ', regex=True)
    return data

In [11]:
#Replace URLs

def replaceURL(df, column):
    data = df.copy()
    data.loc[:, column] = data[column].str.replace('\(http\S+', 'URL', regex=True)
    return data

In [12]:
#Removing symbols

def removeSymbols(df, column):
    data = df.copy()
    data.loc[:, column] = data[column].str.replace('/r/', '', regex=True)
    data.loc[:, column] = data[column].str.replace('[^\.\'A-Za-z0-9]+', ' ', regex=True)
    return data

In [13]:
#Removing numbers

def removeNumbers(df, column):
    data = df.copy()
    data.loc[:, column] = data[column].str.replace('\S*\d\S*', '', regex=True)
    return data

In [14]:
#Remove stop words

stop_words = stopwords.words('english')
stop_words.extend(["ca","http"])
def no_stop_words(list_tokens):    
    tokens_no_stop_word = []
    for elt in list_tokens: 
        tokens_no_stop_word.append([tok for tok in elt if tok not in stop_words])
    return(tokens_no_stop_word)

In [15]:
data = spaceURLs(final_model, 'Title')
data = replaceURL(final_model, 'Title')
data = removeNumbers(final_model, 'Title')
data = removeSymbols(final_model, 'Title')

In [16]:
final_model = data.copy()

In [19]:
final_model.head(10).T

,0,1,2,3,4,5,6,7,8,9
Title,House impeaches Trump for second time over Cap...,I'm a 16 yr old blacksmith and I forged this w...,Do some research before spouting nonsense,Moving boxes arrive at the White House,No One Took Us Seriously Black Cops Warned Ab...,US bans all cotton and tomato products from Xi...,Urban explorer of Reddit what was your I bette...,LPT As a lawyer don't take legal advice from R...,Pirate Bay Founder Thinks Parler s Inability t...,GME YOLO update Jan 13 2021
Upvote_ratio,0.72,0.9,0.9,0.81,0.98,0.88,0.93,0.94,0.88,0.89
Score,39464,29423,50420,149858,7768,112303,30184,32876,79607,37315
Gilded,2,0,0,6,0,0,0,0,2,17
Over_18,False,False,False,False,False,False,True,False,False,False
Number_of_Comments,8801,443,1008,4392,157,4072,3666,1766,3338,3016
word_count,9,18,6,7,15,12,11,11,12,6


# Sentiment Analysis

In [20]:
from textblob import TextBlob

In [21]:
# Create a function to get the subjectivity
def Subjectivity(text):
       return TextBlob(text).sentiment.subjectivity
    
# Create a function to get the polarity
def Polarity(text):
      return  TextBlob(text).sentiment.polarity

In [22]:
final_model['Subjectivity'] = final_model['Title'].apply(Subjectivity)
final_model['Polarity'] = final_model['Title'].apply(Polarity)

In [23]:
final_model.head()

,Title,Upvote_ratio,Score,Gilded,Over_18,Number_of_Comments,word_count,Subjectivity,Polarity
0,House impeaches Trump for second time over Cap...,0.72,39464,2,False,8801,9,0.000000,0.000000
1,I'm a 16 yr old blacksmith and I forged this w...,0.90,29423,0,False,443,18,0.200000,0.100000
2,Do some research before spouting nonsense,0.90,50420,0,False,1008,6,0.000000,0.000000
3,Moving boxes arrive at the White House,0.81,149858,6,False,4392,7,0.000000,0.000000
4,No One Took Us Seriously Black Cops Warned Ab...,0.98,7768,0,False,157,15,0.433333,-0.166667


In [24]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [25]:
senti = SentimentIntensityAnalyzer()
for sentence in final_model['Title']:
    ss = senti.polarity_scores(sentence)

In [26]:
import pickle

In [27]:
pickle.dump(senti, open('senti.pkl','wb'))

In [28]:
#analyzer = SentimentIntensityAnalyzer()
final_model['rating'] = final_model['Title'].apply(senti.polarity_scores)

In [29]:
final_model = pd.concat([final_model.drop(['rating'], axis=1), final_model['rating'].apply(pd.Series)], axis=1)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
# data preprocessing
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def text_preprocess(text):
    text = re.sub(r'[^\w\s]', '', text) 
    l_text = [word for word in text.lower().split() if word not in ENGLISH_STOP_WORDS]
    stem_words = [stemmer.stem(w) for w in l_text]
    lemma_words = [lemmatizer.lemmatize(w) for w in l_text]

    return " ".join(lemma_words)

final_model['Title'] = final_model['Title'].map(lambda x : text_preprocess(x))

In [33]:
final_model.reset_index(inplace=True, drop=True)

# Using the entire dataset to train the model

# Splitting the dataset into features and dependent variable

In [35]:
X = final_model.drop(['Score'], axis = 1)
y = final_model.Score

## One Hot Encoding the over_18 column

In [37]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(X[['Over_18']])
pickle.dump(ohe, open('encoding.pkl','wb'))
col_names = [j for sub in ohe.categories_ for j in sub]

In [38]:
col_names

[False, True]

In [40]:
#Transforming the encoding

trans = ohe.transform(X[['Over_18']])

In [41]:
encoded = pd.DataFrame(trans.todense(), columns=col_names)

<IPython.core.display.Javascript object>

**Converting title column to vector**

In [45]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X['Title'])

Using TensorFlow backend.


In [46]:
word_ind = tokenizer.word_index
word_token = {k:[word_ind[k]] for k in word_ind}
df_word_token = pd.DataFrame(word_token)
df_word_token.to_csv('word_token_final.csv', header=True, index=False)

<IPython.core.display.Javascript object>

In [ ]:
glove_txt = open('glove.6B.100d.txt', encoding="utf8")
embed_dict = dict()
for line in glove_txt:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embed_dict[word] = vector_dimensions
glove_txt.close()

embed_mat_final = np.zeros((len(word_ind) + 1, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embed_dict.get(word)
    if embedding_vector is not None:
        embed_mat_final[index] = embedding_vector

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
embedded_matrix = pd.DataFrame(data=embed_mat_final.astype(float))
embedded_matrix.to_csv('embedded_final.csv', sep = ' ', header =True, float_format='%.2f', index=False)

In [ ]:
max_len = 300

final_title = pad_sequences(tokenizer.texts_to_sequences(X_train["Title"]), padding='post', maxlen=max_len)
train = {}
final_train_title = []
for i, sentence in enumerate(final_title):
    vectors = []
    for n in sentence:
        vectors.append(embed_mat_final[n])
    train[i] = vectors
for key in train:
    arr = np.array(train[key])
    final_train_title.append(np.mean(arr, axis=0))

In [ ]:
final_train_title = pd.DataFrame(np.array(final_train_title))
final_train_title.head()